In [1]:
import scipy.misc
import scipy.signal
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL import ImageFilter
from PIL import ImageChops
from PIL import ImageStat
from scipy.ndimage.filters import gaussian_filter


%matplotlib inline

In [2]:
def Gaussian_Window(sigma,x_pos = 32, y_pos = 32):
    D = np.zeros([64,64])
    for i in range(0,64):
        for j in range(0,64):
            x = i - x_pos
            y = j - y_pos
            r = (1/(2*np.pi*(sigma ** 2)))*np.exp(-(x ** 2 + y ** 2)/(2*(sigma ** 2)))
            D[i][j] = r
    return D/(1/(2*np.pi*(sigma ** 2)))

def Gaussian_mask(sigma,img):
    Gmasked = gaussian_filter(img, sigma=sigma)
    return Gmasked

def r_generate(sigma_m,sigma_w,size=64):
    gW = Gaussian_Window(sigma_w)
    gW = (255.0 / gW.max() * (gW - gW.min())).astype(np.uint8)
    #Image.fromarray(gW,mode='L').save('GW.jpg')
    gNoisse = np.random.normal(loc=0.0, scale=1.0, size=(size,size))
    #gN = (255.0 / gNoisse.max() * (gNoisse - gNoisse.min())).astype(np.uint8)
    gM = Gaussian_mask(2,gNoisse) + 1
    gMimg = (255.0 / gM.max() * (gM - gM.min())).astype(np.uint8)
    Image.fromarray(gMimg,mode='L').save('GM.jpg')
    G = np.multiply(gW,gM)
    G = (255.0 / G.max() * (G - G.min())).astype(np.uint8)
    return G

In [3]:
gW = Gaussian_Window(8,x_pos = 32, y_pos = 32)
gWImg = (255.0 / gW.max() * (gW - gW.min())).astype(np.uint8)
RsImg = Image.fromarray(gWImg,mode='L')
RsImg.save('GaussianWindow.jpg')

In [4]:
r = r_generate(2,8)
rImg = Image.fromarray(r,mode='L')
rImg.save('rrr.jpg')

In [5]:
R = np.zeros((2000,4096))
for i in range(0,2000):
    r = np.ravel(r_generate(2,8))
    R[i] = r


In [182]:
W = np.random.uniform(low=-0.1, high=0.1, size=(16,4096))

In [183]:
Y = np.dot(W,R[0])
Y.reshape((16,1))
np.tril(Y*np.transpose(Y)).shape

(16, 16)

In [206]:
beta = 10000
dW = np.ones((4096,16))
X = R[0].reshape((4096,1))
W /= np.linalg.norm(W, axis=1).reshape((16,1))
for i in range(0,2000):
    Y = Y.reshape((16,1))
    X = R[i].reshape((4096,1))
    dW = beta*(np.dot(Y,np.transpose(X))- np.dot(np.tril(Y*np.transpose(Y)),W))
    dW /= np.linalg.norm(dW, axis=1).reshape((16,1))
    W /= np.linalg.norm(W, axis=1).reshape((16,1))
    W = W + dW
    
    Y = np.dot(W,X)

In [215]:
Y_W = np.empty((16,64,64))
yw = 0
for w in W:
    Y_W[yw] = w.reshape((64,64))
    yw += 1

(64, 64)

In [229]:
iW0 = (255.0 / (Y_W[0]+1).max() * ((Y_W[0]+1) - (Y_W[0]+1).min())).astype(np.uint8)
iw0 = Image.fromarray(iW0,mode='L')
iw0.save('./weights/W00p.jpg')

In [232]:
for i in range(0,16):
    z = str(i).zfill(4)
    mmm = np.abs(Y_W[i].min()) 
    iW = (255.0 / (Y_W[i]+mmm).max() * ((Y_W[i]+mmm) - (Y_W[i]+mmm).min())).astype(np.uint8)
    Image.fromarray(iW,mode='L').save('./weights/W'+z+'.jpg')